# Reading and Visualizing GeoJSON Data

In [10]:
# Standard libraries
import json

# Third-party libraries for data manipulation
import pandas as pd
import geopandas as gpd

# Libraries for MongoDB
from pymongo import MongoClient
import pymongo

# Libraries for visualization
import folium
from IPython.display import display, clear_output, HTML, Javascript

# Libraries for interactive widgets
from ipywidgets import widgets, interact, HTML

In [11]:
# Load the MongoDB connection string from the JSON file
PATH_TO_SECRET_JSON = '/home/jovyan/keys/mongodb_key.json'
with open(PATH_TO_SECRET_JSON) as f:
    MONGODB_URI = json.load(f)['connection_string']

# Extract the database name from the connection string
db_name = MONGODB_URI.split('/')[-1].split('?')[0]

# Create a MongoDB client using the connection string
if MONGODB_URI:
    client = MongoClient(MONGODB_URI)
    print("The MongoDB client has been initialized.")
else:
    print("Failed to initialize the MongoDB client.")

# Print the version of the pymongo package
pymongo_version = pymongo.__version__
print(f"The version of the pymongo package is {pymongo_version}")

# Connect to the specified MongoDB database and collection
db = client[db_name]
collection = db.attractions

The MongoDB client has been initialized.
The version of the pymongo package is 4.4.1


In [12]:
# Read the GeoJSON file into a GeoDataFrame
gdf = gpd.read_file("../data/neighborhoods/nycneighborhoods.geojson.json")
unique_boroughs = gdf['borough'].unique()

# Functional

In [13]:
# Initialize the HTML widget with empty table headers
def initialize_html_widget():
    table_widget = HTML(value='<table><tr><th>Latitude</th><th>Longitude</th></tr></table>')
    return table_widget

table_widget = initialize_html_widget()

In [14]:
from ipywidgets import Dropdown

# Create a dropdown widget for boroughs
def initialize_borough_dropdown(unique_boroughs):
    borough_dropdown = Dropdown(
        options=unique_boroughs,
        value=unique_boroughs[0],  # Default value
        description='Borough:'
    )
    return borough_dropdown

# Initialize the HTML widget with empty table headers
def initialize_html_widget():
    table_widget = HTML(value='<table><tr><th>Latitude</th><th>Longitude</th></tr></table>')
    return table_widget

# Get the unique boroughs from the GeoDataFrame
unique_boroughs = gdf['borough'].unique()

# Initialize the dropdown and table widgets
borough_dropdown = initialize_borough_dropdown(unique_boroughs)
table_widget = initialize_html_widget()

# Display the widgets as a test
#display(borough_dropdown)
#display(table_widget)


In [16]:
from IPython.display import display, clear_output, Javascript
import folium

borough_dropdown = widgets.Dropdown(
    options=unique_boroughs,
    value=unique_boroughs[0],  # or 'Manhattan' if it's in the list
    description='Borough:',
)


# Function to display boundary coordinates in the table widget
def display_boundary_points(neighborhood_name):
    # Dummy coordinates for demonstration
    boundary_coords = [(40, -73), (41, -72)]  
    
    table_content = '<table><tr><th>Latitude</th><th>Longitude</th></tr>'
    for x, y in boundary_coords:
        table_content += f"<tr><td>{y}</td><td>{x}</td></tr>"
    table_content += '</table>'
    
    table_widget.value = table_content

# Function to update the map
def update_map(borough):
    clear_output(wait=True)
    
    # Display the dropdown widget
    display(borough_dropdown)
    
    # Filter the GeoDataFrame by the selected borough
    gdf_filtered = gdf[gdf['borough'] == borough]
    
    # Center the map based on the filtered GeoDataFrame
    x_center = gdf_filtered.geometry.unary_union.centroid.x
    y_center = gdf_filtered.geometry.unary_union.centroid.y
    m = folium.Map(location=[y_center, x_center], zoom_start=13)
    
    # Add markers and popups
    for idx, row in gdf_filtered.iterrows():
        x = row['geometry'].centroid.x
        y = row['geometry'].centroid.y
        neighborhood_name = row['neighborhood']
        
        html = f"""
        <h4>{neighborhood_name}</h4>
        <button onclick="console.log('Button clicked for {neighborhood_name}'); var kernel = IPython.notebook.kernel; kernel.execute('display_boundary_points(\\'{neighborhood_name}\\')')">Show Coordinates</button>
        """

        popup = folium.Popup(html, max_width=400)
        folium.Marker([y, x], popup=popup).add_to(m)
    
    # Show the map and table widget
    display(m)
    display(table_widget)

# Link the dropdown widget to the update_map function
borough_dropdown.observe(lambda change: update_map(change['new']), names='value')

# Display the initial map
update_map(unique_boroughs[0])


Dropdown(description='Borough:', index=2, options=('Bronx', 'Queens', 'Staten Island', 'Brooklyn', 'Manhattan'…

HTML(value='<table><tr><th>Latitude</th><th>Longitude</th></tr></table>')